In [3]:
import random
import math
# from optimisation.python.models.conic_model import single_mosek
# from optimisation.python.models.approximation_model import approx_model
# from optimisation.python.utils import *

In [ ]:
def healthprob(pool, pop):
    """Compute the probability that the test result of the given pool is negative (i.e., healthy)."""
    healthprob = 1
    for i in pool:
        healthprob *= pop[i][0]
    return healthprob

In [ ]:
def utilsum(pool, pop):
    """Compute the sum of the utilities pool."""
    return sum([pop[i][1] for i in pool])

In [ ]:
def welfare(pool, pop):
    """Compute the (expected) welfare of the given pool."""
    return healthprob(pool, pop)*utilsum(pool, pop)

In [ ]:
def pop2vec(pop):
    keylist = list(pop.keys())
    q = [pop[k][0] for k in keylist]
    u = [pop[k][1] for k in keylist]
    return q, u, keylist

In [ ]:
def solve_conic(pop, G):
    """Solves the optimal allocation of a single test of size at most G 
    using an exponential cone optimisation solver."""
    if not pop:
        return 0, {}
    q, u, keylist = pop2vec(pop)  # Get input vectors for model

    w, x = single_mosek(q, u, G)
    p = [[i+1 for i in range(len(q)) if x[i] == 1]]
    pool = [[keylist[i-1] for i in pool] for pool in p][0]

    w = welfare(pool, pop)
    return w, pool

In [5]:
def greedy(pop, T, G=N):

    def remove_zeros(pop):
        """Remove people with zero (or negative) utilities from the population. Returns new population dict."""
        return {p: val for p, val in pop.items() if val[1] > 0}

    """
    Provides a greedy, non-overlapping solution to the daily allocation problem for population `pop`.

    The algorithm computes a greedy solution using as a subroutine an exponential cone optimisation 
    that estimates the optimal allocation of a single test.

    Inputs
    ------
    pop::Dict - maps every person's ID to tuple (q,u) (health probabilities and utilities)
    T::Int 	  - number of tests (test budget)
    G::Int    - pooled test size

    Output
    ------
    Dict - mapping from pool names (letters) to pools (integer arrays).
    """

    if not pop:
        return 0, {}
    pop = remove_zeros(pop)
    welfares, pools = [], []
    for t in range(T):
        w, pool = solve_conic(pop, G)
        welfares.append(w) # record welfare
        pools.append(pool) # record pool
        # Remove people in pool from population
        pop = {p: val for p, val in pop.items() if p not in pool}
    named_pools = { chr(65+i) : pool for i, pool in enumerate(pools) }
    return sum(welfares), named_pools